### Theoretical Oscillation models

This notebook expose the method to apply a *Depthwise separable convolutional NN* on the oscilation star observation.

In [1]:
#library(devtools)
#install_github("rmaestre/variableStars", ref="oscillationCodes")
library(variableStars)
library(data.table)
library(ggplot2)
library(RColorBrewer)
library(plotly)
library(keras)
library(plotly)
library(abind)
library(fields)
library(doParallel)


Attaching package: 'plotly'

The following object is masked from 'package:ggplot2':

    last_plot

The following object is masked from 'package:stats':

    filter

The following object is masked from 'package:graphics':

    layout

Loading required package: spam
Loading required package: dotCall64
Loading required package: grid
Spam version 2.2-0 (2018-06-19) is loaded.
Type 'help( Spam)' or 'demo( spam)' for a short introduction 
and overview of this package.
Help for individual functions is also obtained by adding the
suffix '.spam' to the function name, e.g. 'help( chol.spam)'.

Attaching package: 'spam'

The following objects are masked from 'package:base':

    backsolve, forwardsolve

Loading required package: maps
See www.image.ucar.edu/~nychka/Fields for
 a vignette and other supplements. 
Loading required package: foreach
Loading required package: iterators
Loading required package: parallel


### Global parameters

In [2]:
# Number of rows per batch training
experiment_number <- 100
# Resolution for target frequency [0-100]
resolution <- 0.5

# Input dimension
cuts_breaks <- c(-Inf, seq(0, 101, resolution), Inf)
input_dim <- length(cuts_breaks) - 1

# Output dimension
num_classes <-
  length(seq(
    from = 0.1,
    to = 14 / 0.0864,
    by = 1
  )) # Buckets of possible classes

normalized <- function(x) {
        (x - min(x)) / (max(x) - min(x))
      }

### Auxiliar functions

In [3]:
trunc <-
  function(x, ..., prec = 1)
    base::trunc(x * 10 ^ prec, ...) / 10 ^ prec


c_dnu <- function(x) {
    length(which(x == 1)) == 1
  }
  c_dr <- function(x) {
    length(which(x == 2)) == 1
  }
  c_over <- function(x) {
    length(which(x == 3)) == 1
  }

flat <- function(x) {
  return(paste0(trunc(c(x), prec=4), collapse = ","))
}

### File processing

In [4]:
process_full_file <- function(full_file_path, resolution, cuts_breaks) {
library(fields)
library(variableStars)
# Max n_mode
  max_n <- 10
  max_l <- 3
  
  # Read file
  data = read.csv(
    full_file_path,
    header = FALSE,
    sep = "",
    skip = 25,
    stringsAsFactors = FALSE,
    col.names = c("n", "l", "m", "nu", "f", "no",
                  "pc", "i0")
  )
  
  # Transform fequencies
  data$nu <-  data$nu * 0.0864
  
  # Drop n modes over max_n
  data <- data[data$n < max_n, ]
  # Drop l modes over max_n
  data <- data[data$l < max_l, ]
  # Keep ony m = 0
  data <- data[data$m == 0, ]
    
  if (nrow(data) != 0) {
    # DR estimation
    dr <-
      as.numeric(strsplit(readLines(
        file(full_file_path, "r"),
        n = 15,
        skipNul = T
      )[14:14], "\\s+")[[1]][4])
    dr <- dr * 0.0864
    
    # DNU estimation
    
    dnu <-
      mean(aggregate(nu ~ l, data = data, function(x)
        median(diff(x)))$nu)
    
    
    # Generate random frequencies
    data$amp <- runif(length(data$nu), 0, 1)
        
    
    chunks <- unlist(strsplit(full_file_path, "/"))
    filename <- chunks[length(chunks)]
    filename_chunks <- unlist(strsplit(filename, ".frq"))
    
    
    file_output <-
      paste0("~/Downloads/data/", filename_chunks, ".log")
    
    # Execute experiment
    result <- process(
      frequency = data$nu,
      amplitude = data$amp,
      filter = "uniform",
      gRegimen = 0,
      maxDnu = 1,
      minDnu = 15,
      numFrequencies = 30,
      dnuGuessError = -1,
      debug = F,
      processFirstRangeOnly = 30
    )
    
    # X data. THe maximum value is processed in each bucket
    # ----------------------
    # Save fourier transform
    ftS <-
      stats.bin(as.numeric(result$fresAmps[[names(result$fresAmps)[1]]]$fInv),
                as.numeric(result$fresAmps[[names(result$fresAmps)[1]]]$b),
                breaks = cuts_breaks)$stats
    ft_1D <- ftS[8, 1:(length(cuts_breaks) - 1)]
    ft_1D[is.na(ft_1D)] <- 0
    
    
    # Save histogram of diffs
    diffS <-
      stats.bin(
        as.numeric(result$diffHistogram$histogram$bins),
        as.numeric(result$diffHistogram$histogram$values),
        breaks = cuts_breaks
      )$stats
    diff_2D <- diffS[8, 1:(length(cuts_breaks) - 1)]
    diff_2D[is.na(diff_2D)] <- 0
    
    # Save crosscorrelation
    cross <- stats.bin(
      as.numeric(result$crossCorrelation$index),
      as.numeric(result$crossCorrelation$autocorre),
      breaks = cuts_breaks
    )$stats
    cross_3D <- cross[8, 1:(length(cuts_breaks) - 1)]
    cross_3D[is.na(cross_3D)] <- 0
    
    # Assert all dimensions are equal
    stopifnot((length(ft_1D) == length(diff_2D)) ==
                ((length(diff_2D) == length(cross_3D)) ==
                   (
                     length(cross_3D) == length(cuts_breaks) - 1
                   )))
    
    # Write to disk
    write(paste(
      flat(normalized(ft_1D)),
      flat(normalized(diff_2D)),
      flat(normalized(cross_3D)),
      flat(c(dr, dnu)),
      sep = ","
    )
    ,
    file = file_output,
    append = F)
    
    
  } else {
    print(paste0("Empty file:", paste0(full_dir, "/", file)))
  }
}

### Parallel processing

In [5]:
all_files <- c()
base_dir <- "/home/roberto/Downloads/evolutionTracks/FILOU/"
setwd(base_dir)
dirs <- list.dirs(recursive = T)
# Loop over files
setwd(base_dir)
for (dir in dirs[grepl("*VO*", list.dirs(recursive = T))]) {
    full_dir <- paste0(base_dir, basename(dir))
    setwd(full_dir)
    if (!is.na(basename(dir))) {
    #print(paste0("Processing directory: ", full_dir))
    # Change directory work
    setwd(full_dir)
        for (file in list.files(pattern = "*frq")) {
            all_files <- c(all_files, paste0(full_dir,"/",file))
        }
    }
}

In [6]:
head(all_files)
length(all_files)

[1] "/home/roberto/Downloads/evolutionTracks/FILOU/VO-m130fe-2a164o0rotjpzt5p5-ad/00000-m130fe-2a164o0rotjpzt5p5-ad.frq"
[2] "/home/roberto/Downloads/evolutionTracks/FILOU/VO-m130fe-2a164o0rotjpzt5p5-ad/00001-m130fe-2a164o0rotjpzt5p5-ad.frq"
[3] "/home/roberto/Downloads/evolutionTracks/FILOU/VO-m130fe-2a164o0rotjpzt5p5-ad/00002-m130fe-2a164o0rotjpzt5p5-ad.frq"
[4] "/home/roberto/Downloads/evolutionTracks/FILOU/VO-m130fe-2a164o0rotjpzt5p5-ad/00003-m130fe-2a164o0rotjpzt5p5-ad.frq"
[5] "/home/roberto/Downloads/evolutionTracks/FILOU/VO-m130fe-2a164o0rotjpzt5p5-ad/00004-m130fe-2a164o0rotjpzt5p5-ad.frq"
[6] "/home/roberto/Downloads/evolutionTracks/FILOU/VO-m130fe-2a164o0rotjpzt5p5-ad/00005-m130fe-2a164o0rotjpzt5p5-ad.frq"

[1] 524759

In [7]:
cl <- parallel::makeCluster(7)
doParallel::registerDoParallel(cl)

In [8]:
foreach(i = all_files, .combine = 'c') %dopar% {
 # Get destination file
 chunks <- unlist(strsplit(as.character(i), "/"))
 filename <- chunks[length(chunks)]
 filename_chunks <- unlist(strsplit(filename, ".frq"))
 if(!file.exists(paste0("~/Downloads/data/", filename_chunks, ".log"))){
  process_full_file(as.character(i), resolution, cuts_breaks)
 }
}

[1] "Empty file:/home/roberto/Downloads/evolutionTracks/FILOU/VO-m300fe0a164o0rotjpzt5p7-ad/00660-m300fe0a164o0rotjpzt5p7-ad.frq"
  [2] "Empty file:/home/roberto/Downloads/evolutionTracks/FILOU/VO-m300fe0a164o0rotjpzt5p7-ad/00660-m300fe0a164o0rotjpzt5p7-ad.frq"
  [3] "Empty file:/home/roberto/Downloads/evolutionTracks/FILOU/VO-m300fe0a164o0rotjpzt5p7-ad/00660-m300fe0a164o0rotjpzt5p7-ad.frq"
  [4] "Empty file:/home/roberto/Downloads/evolutionTracks/FILOU/VO-m300fe0a164o0rotjpzt5p7-ad/00660-m300fe0a164o0rotjpzt5p7-ad.frq"
  [5] "Empty file:/home/roberto/Downloads/evolutionTracks/FILOU/VO-m300fe0a164o0rotjpzt5p7-ad/00660-m300fe0a164o0rotjpzt5p7-ad.frq"
  [6] "Empty file:/home/roberto/Downloads/evolutionTracks/FILOU/VO-m300fe0a164o0rotjpzt5p7-ad/00660-m300fe0a164o0rotjpzt5p7-ad.frq"
  [7] "Empty file:/home/roberto/Downloads/evolutionTracks/FILOU/VO-m300fe0a164o0rotjpzt5p7-ad/00660-m300fe0a164o0rotjpzt5p7-ad.frq"
  [8] "Empty file:/home/roberto/Downloads/evolutionTracks/FILOU/VO-m300fe0a164o0rotjpzt5p7-ad/00660-m300fe0a164o0rotjpzt5p7-ad.frq"
  [9] "Empty file:/home/roberto/Downloads/evolutionTracks/FILOU/VO-m300fe0a164o0rotjpzt5p7-ad/00660-m300fe0a164o0rotjpzt5p7-ad.frq"
 [10] "Empty file:/home/roberto/Downloads/evolutionTracks/FILOU/VO-m300fe0a164o0rotjpzt5p7-ad/00660-m300fe0a164o0rotjpzt5p7-ad.frq"
 [11] "Empty file:/home/roberto/Downloads/evolutionTracks/FILOU/VO-m300fe0a164o0rotjpzt5p7-ad/00660-m300fe0a164o0rotjpzt5p7-ad.frq"
 [12] "Empty file:/home/roberto/Downloads/evolutionTracks/FILOU/VO-m300fe0a164o0rotjpzt5p7-ad/00660-m300fe0a164o0rotjpzt5p7-ad.frq"
 [13] "Empty file:/home/roberto/Downloads/evolutionTracks/FILOU/VO-m300fe0a164o0rotjpzt5p7-ad/00660-m300fe0a164o0rotjpzt5p7-ad.frq"
 [14] "Empty file:/home/roberto/Downloads/evolutionTracks/FILOU/VO-m300fe0a164o0rotjpzt5p7-ad/00660-m300fe0a164o0rotjpzt5p7-ad.frq"
 [15] "Empty file:/home/roberto/Downloads/evolutionTracks/FILOU/VO-m300fe0a164o0rotjpzt5p7-ad/00660-m300fe0a164o0rotjpzt5p7-ad.frq"
 [16] "Empty file:/home/roberto/Downloads/evolutionTracks/FILOU/VO-m300fe0a164o0rotjpzt5p7-ad/00660-m300fe0a164o0rotjpzt5p7-ad.frq"
 [17] "Empty file:/home/roberto/Downloads/evolutionTracks/FILOU/VO-m300fe0a164o0rotjpzt5p7-ad/00660-m300fe0a164o0rotjpzt5p7-ad.frq"
 [18] "Empty file:/home/roberto/Downloads/evolutionTracks/FILOU/VO-m300fe0a164o0rotjpzt5p7-ad/00660-m300fe0a164o0rotjpzt5p7-ad.frq"
 [19] "Empty file:/home/roberto/Downloads/evolutionTracks/FILOU/VO-m300fe0a164o0rotjpzt5p7-ad/00660-m300fe0a164o0rotjpzt5p7-ad.frq"
 [20] "Empty file:/home/roberto/Downloads/evolutionTracks/FILOU/VO-m300fe0a164o0rotjpzt5p7-ad/00660-m300fe0a164o0rotjpzt5p7-ad.frq"
 [21] "Empty file:/home/roberto/Downloads/evolutionTracks/FILOU/VO-m300fe0a164o0rotjpzt5p7-ad/00660-m300fe0a164o0rotjpzt5p7-ad.frq"
 [22] "Empty file:/home/roberto/Downloads/evolutionTracks/FILOU/VO-m300fe0a164o0rotjpzt5p7-ad/00660-m300fe0a164o0rotjpzt5p7-ad.frq"
 [23] "Empty file:/home/roberto/Downloads/evolutionTracks/FILOU/VO-m300fe0a164o0rotjpzt5p7-ad/00660-m300fe0a164o0rotjpzt5p7-ad.frq"
 [24] "Empty file:/home/roberto/Downloads/evolutionTracks/FILOU/VO-m300fe0a164o0rotjpzt5p7-ad/00660-m300fe0a164o0rotjpzt5p7-ad.frq"
 [25] "Empty file:/home/roberto/Downloads/evolutionTracks/FILOU/VO-m300fe0a164o0rotjpzt5p7-ad/00660-m300fe0a164o0rotjpzt5p7-ad.frq"
 [26] "Empty file:/home/roberto/Downloads/evolutionTracks/FILOU/VO-m300fe0a164o0rotjpzt5p7-ad/00660-m300fe0a164o0rotjpzt5p7-ad.frq"
 [27] "Empty file:/home/roberto/Downloads/evolutionTracks/FILOU/VO-m300fe0a164o0rotjpzt5p7-ad/00660-m300fe0a164o0rotjpzt5p7-ad.frq"
 [28] "Empty file:/home/roberto/Downloads/evolutionTracks/FILOU/VO-m300fe0a164o0rotjpzt5p7-ad/00660-m300fe0a164o0rotjpzt5p7-ad.frq"
 [29] "Empty file:/home/roberto/Downloads/evolutionTracks/FILOU/VO-m300fe0a164o0rotjpzt5p7-ad/00660-m300fe0a164o0rotjpzt5p7-ad.frq"
 [30] "Empty file:/home/roberto/Downloads/evolutionTracks/FILOU/VO-m300fe0a164o0rotjpzt5p7-ad/00660-m300fe0a164o0rotjpzt5p7-ad.frq"
 [31] "Empty file:/home/roberto/Downloads/

In [9]:
parallel::stopCluster(cl)